在Lovelive SIF中卡牌技能的效果有加分、回hp、大幅度强化判定、小幅度强化判定四种，其中前两种效果是瞬发的，而强化判定的效果则会持续一段时间
由于4.0版本以后强化宝石的引入，拥有判定技能的卡牌在强化判定的效果期间有33%的得分加成，因此在考虑卡牌强度的时候我们需要计算卡牌的“等效强度”，对于判卡来说，覆盖率是一项重要指标。
在游戏过程中，卡组的9张卡中可能有多张判卡使得情况变得非常复杂，本文着重研究单张判卡的覆盖率

问题假设：在没有判定强化的前提下，玩家每一个note打出的判定互相独立并且服从分布
$$ \mathcal{P} = [ p_{Perfect}, p_{Great}, p_{Good}, p_{Bad}, p_{Miss} ] $$

对于某一个谱面，我们知道曲目时长$T$和note总数$N$，以及每个note到达的时间$ \{ t_i \}_{i=1}^N $。
以N判为例，假设技能描述为“每K个note，以$p$的概率在接下来的$D$秒内加强判定”，那么对于每一个note，note数增加的概率为
$$ \alpha = p_{Perfect} + p_{Great} + p_{Good} + p_{Bad} $$

设$x_i$为第$i$个note打出的判定等级，$d_i$为第i个note是否被技能覆盖的指示变量，设$ \gamma_i = \{ t_j \}_{j=1}^{i} \cup \{ x_j \}_{j=1}^{i-1} \cup \{d_j\}_{j=1}^{i-1} $，即在第i个note被按下之前的所有已知历史，根据条件期望的chain rule，我们可以把覆盖率可以写成
$$ 覆盖率 = \frac{1}{N} \mathbb{E}[\sum_{i=1}^N d_i] = \frac{1}{N} \mathbb{E}[ \mathbb{E}[ d_1 + \mathbb{E}[ d_2 + \cdots \mathbb{E}[ d_{N-1} + \mathbb{E}[ d_N | \gamma_N ] | \gamma_{N-1} ] \cdots | \gamma_3 ] | \gamma_2 ] | \gamma_1 ] $$

不幸的是，以上问题过于复杂并没有解析解，为了简化问题，我们需要追加如下假设：
* 所有的长note都以普通note看待
* note到达的时间间隔是相等的，即 $t_i = i \Delta t$
* 强化状态的持续时间$D$是$\Delta t$的整数倍，在接下来的推导中设$ \tilde{D} = D / \Delta t $

首先我们考虑N判技能:“每K个note，以$p$的概率在接下来的$D$秒内加强判定”，在追加假设下可以简化为“每K个note，以$p$的概率触发技能，每当技能被触发，那么为接下来的$\tilde{D}$个note加强判定”。
我们可以建立一个马尔可夫链，状态和状态空间为
$$ (c,h,r) \in \Omega = \{0,1,\ldots,K-1\} \times \{0,1\} \times \{0,1,\ldots,\tilde{D}\} $$
* $c$ - 为当前累计的note数，每当累积到$K$个会根据下图判断是否累计继承或触发技能
* $h$ - 为累计继承的指示变量，1代表当前已经有累计继承，反之为0
* $r$ - 为当前技能加强状态剩余的时间(剩余的note数)

<img src="https://www.lovelivewiki.com/images/0/09/%E5%9B%BE4.1.png" width="500">
根据上图（来自《判定覆盖率和判卡培养》），列出所有可能的状态转移

输入状态$(c,h,r)$, 首先计算剩余时间 $ \tilde{r} = \max(0,r-1) $

1. 无时间覆盖$\tilde{r}=0$，无累积继承$h=0$:

    * 如果下一个note是Miss: 
    $ \Pr[(c,0,r) \to (c,0,0)] = 1-\alpha $
    * 如果下一个note不是Miss: 
        * 如果未达到累积计数$c<K-1$: 
        $ \Pr[(c,0,r) \to (c+1,0,0)] = \alpha $
        * 如果达到累积计数$c=K-1$: 
            * 如果技能被触发: 
            $ \Pr[(K-1,0,r) \to (0,0,\tilde{D})] = p \alpha $
            * 如果技能未被触发: 
            $ \Pr[(K-1,0,r) \to (0,0,0)] = (1-p) \alpha $
            
2. 无时间覆盖$\tilde{r}=0$，有累积继承$h=1$:

    * 如果下一个note是Miss: 
        * 如果技能被触发:
        $ \Pr[(c,1,r) \to (c,0,\tilde{D})] = p (1-\alpha) $
        * 如果技能未被触发:
        $ \Pr[(c,1,r) \to (c,0,0)] = (1-p) (1-\alpha) $
    * 如果下一个note不是Miss: 
        * 如果未达到累积计数$c<K-1$:
            * 如果技能被触发: 
            $ \Pr[(c,1,r) \to (c+1,0,\tilde{D})] = p \alpha $
            * 如果技能未被触发: 
            $ \Pr[(c,1,r) \to (c+1,0,0)] = (1-p) \alpha $
        * 如果达到累积计数$c=K-1$:
            * 如果技能被触发: 
            $ \Pr[(K-1,1,r) \to (0,1,\tilde{D})] = p \alpha $
            * 如果技能未被触发: 
            $ \Pr[(K-1,1,r) \to (0,1,0)] = (1-p) \alpha $

3. 有时间覆盖$\tilde{r}>0$
    * 如果下一个note是Miss: 
    $ \Pr[(c,h,r) \to (c,h,\tilde{r})] = 1-\alpha $
    * 如果下一个note不是Miss: 
        * 如果未达到累积计数$c<K-1$:
        $ \Pr[(c,h,r) \to (c+1,h,\tilde{r})] = \alpha $
        * 如果达到累积计数$c=K-1$ (注:如此时$h=1$则发生自重合损失):
        $ \Pr[(K-1,h,r) \to (0,1,\tilde{r})] = \alpha $

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
def next_state_simul(state,K,D,alpha,p):
	combo, inherit, remain = state
	remain = max(remain-1,0)
	
	if remain == 0 and inherit == 0:
		if np.random.random() < 1-alpha:
			return (combo,0,0), 1 - alpha
		else:
			if combo < K-1:
				return (combo+1,0,0), alpha
			elif combo == K-1:
				if np.random.random() < p:
					return (0,0,D), p*alpha
				else:
					return (0,0,0), (1-p)*alpha
	elif remain == 0 and inherit == 1:
		if np.random.random() < 1-alpha:
			if np.random.random() < p:
				return (combo,0,D), p*(1 - alpha)
			else:
				return (combo,0,0), (1-p)*(1 - alpha)
		else:
			if combo < K-1:
				if np.random.random() < p:
					return (combo+1,0,D), p*alpha
				else:
					return (combo+1,0,D), (1-p)*alpha
			elif combo == K-1:
				if np.random.random() < p:
					return (0,1,D), p*alpha
				else:
					return (0,1,0), (1-p)*alpha
	elif remain > 0:
		if np.random.random() < 1-alpha:
			return (combo,inherit,remain), 1 - alpha
		else:
			if combo < K-1:
				return (combo+1,inherit,remain), alpha
			elif combo == K-1:
				return (0,1,remain), alpha

def next_state(state,K,D,alpha,p):
	combo, inherit, remain = state
	remain = max(remain-1,0)
	new_state = dict()
	if remain == 0 and inherit == 0:
		new_state[(combo,0,0)] = 1 - alpha
		if combo < K-1:
			new_state[(combo+1,0,0)] = alpha
		elif combo == K-1:
			new_state[(0,0,D)] = p*alpha
			new_state[(0,0,0)] = (1-p)*alpha
	elif remain == 0 and inherit == 1:
		new_state[(combo,0,D)] = p*(1 - alpha)
		new_state[(combo,0,0)] = (1-p)*(1 - alpha)
		if combo < K-1:
			new_state[(combo+1,0,D)] = p*alpha
			new_state[(combo+1,0,0)] = (1-p)*alpha
		elif combo == K-1:
			new_state[(0,1,D)] = p*alpha
			new_state[(0,1,0)] = (1-p)*alpha
	elif remain > 0:
		new_state[(combo,inherit,remain)] = 1 - alpha
		if combo < K-1:
			new_state[(combo+1,inherit,remain)] = alpha
		elif combo == K-1:
			new_state[(0,1,remain)] = alpha
	return new_state
def generate_Markov(K,D,alpha,p):
	queue, trans = [(0,0,0)], dict()
	while len(queue)>0:
		state = queue.pop()
		if trans.get(state) is None:
			to_states = next_state(state,K,D,alpha,p)
			trans[state] = dict()
			for st,pr in to_states.items():
				if pr >= 0:
					trans[state][st] = pr
					queue = [st]+queue
	G = nx.DiGraph()
	# Use ASCII character instead of number to make node name shorter
	for from_state, sub_dict in trans.items():
		for to_state, prob in sub_dict.items():
			G.add_edge('{0}{1}{2}'.format(*[chr(65+x) for x in from_state]), 
					   '{0}{1}{2}'.format(*[chr(65+x) for x in to_state]), prob=prob)
	M = np.array(nx.adjacency_matrix(G, weight='prob', nodelist=sorted(list(G.node))).todense(), dtype=float)
	state_list = sorted(list(G.node))
	return M, state_list

给定以上状态转移的概率，从状态$(0,0,0)$开始使用广度优先算法遍历所有可以达到的状态以及状态转移矩阵$M$
这个马尔可夫链是无周期不可分解的（至少在$\alpha<1$的情况下），这意味着存在唯一的静态分布$\pi$满足
$$ \sum_{\omega \in \Omega} \pi(\omega) = 1, \qquad \pi(\omega) = \sum_{\omega^\prime \in \Omega} M_{\omega^\prime \omega} \pi(\omega^\prime), \quad \forall \, \omega \in \Omega $$

例如假设玩家打出Miss的概率为0.01，并且卡牌技能简化后等价为“每3个note，以$40\%$的概率触发技能，每当技能被触发，那么为接下来的$4$个note加强判定”，那么对应马尔可夫链的转移矩阵将如下图所示

In [2]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300
K, D_tilde, alpha, p = 3, 5, 1, 0.4
M, state_list = generate_Markov(K,D_tilde,alpha,p)
df = pd.DataFrame(M)
df.index = [''.join([str(ord(x)-ord('A')) for x in name]) for name in state_list]
df.columns = [''.join([str(ord(x)-ord('A')) for x in name]) for name in state_list]
df.style.apply(lambda x: ["background: rgb(255,{0},128)".format(255-int(255*v)) if v > 0 else "" for v in x], axis = 1)

判定强化的存在等同于判定强化的状态剩余时间大于0，所有判定强化持续情况下的状态的集合可以表示为
$$ \Omega^* = \{ (c,h,r) \in \Omega \,|\, r > 0 \} $$

我们感兴趣的覆盖率可以近似为马尔可夫链处于判定强化持续情况下的概率，因此
$$ 近似覆盖率1 = \sum_{\omega \in \Omega^*} \pi(\omega) $$

值得注意的是，静态分布$\pi$一般一般描述马尔可夫链长期运行后的行为，因此不依赖于初始状态。
但是游戏中，初始状态一定为$(0,0,0)$，并且note个数在50-1800这个范围之间，因此上式实际是真实覆盖率的上界。

为了更准确地计算覆盖率，可以定义$\pi^{(0)} = [1,0,\ldots,0]$，对于$i=1,\ldots,N$, $ \pi^{(i)} = \pi^{(i-1)} M $
$$ 近似覆盖率2 = \frac{1}{N} \sum_{i=1}^N \sum_{\omega \in \Omega^*} \pi^{(i)}(\omega) $$

正确性验证: 这里我们对比《判定覆盖率和判卡培养》一文中已有相对精确的覆盖率数据和前文提出的两个覆盖率近似公式，并且假设玩家打出miss的概率为0.001

In [3]:
import urllib.request
url = 'https://www.lovelivewiki.com/w/%E5%88%A4%E5%AE%9A%E8%A6%86%E7%9B%96%E7%8E%87%E5%92%8C%E5%88%A4%E5%8D%A1%E5%9F%B9%E5%85%BB'
df_total = pd.read_html(urllib.request.urlopen(url))

In [5]:
get_num = lambda x: float(x[:-1])/100
get_pct = lambda x: '{0:.4f}%'.format(x*100)
df_list, alpha = [], 1
for idx, num_note, time in zip([-3,-2,-1], [722, 584, 383], [110, 110, 88]):
	df = df_total[idx].iloc[2:]
	df.index = list(range(0,len(df)))
	# CR - cover rate, SCLR - self-covering loss rate
	df.columns = ['Card', 'note', 'prob', 'duration', 'CR', 'SCLR']
	df = pd.concat([df[['Card', 'note', 'prob', 'duration', 'CR']],
					pd.DataFrame(columns=['Approx CR 1', 'Approx CR 2'])], axis=1)
	
	note_density = float(df_total[idx].iloc[0,3])
	note_density = num_note/time
	for i, item in df.iterrows():
		K, p, D = item[1:4]
		K, p, D_tilde = int(K), float(p)/100, round(note_density*float(D))
		M, state_list = generate_Markov(K,D_tilde,alpha,p)
		print('Computing {0} out of {1}, {2} states in the Markov chain'.format(i,len(df),len(state_list)))
		# Compute CR by stationary distribution
		ns = M.shape[0]
		A = M - np.eye(ns)
		A[:,-1] = 1
		eq_dist = np.array([0]*(ns-1)+[1]).reshape((1,-1)) @ np.linalg.inv(A)
		idx_no_cover = np.array([i for i,x in enumerate(state_list) if x[-1]=='A'])
		cover_rate1 = 1 - eq_dist[0,idx_no_cover].sum()
		df.iloc[i,-2] = get_pct(cover_rate1)
		# Compute CR by Chapman-Kolmogorov
		x = np.array([1.0]+[0.0]*(ns-1)).reshape((1,-1))
		cum_prob = 0
		for t in range(num_note):
			cum_prob += 1-x[0,idx_no_cover].sum()
			x = x @ M
		cover_rate2 = cum_prob/num_note
		df.iloc[i,-1] = get_pct(cover_rate2)
		
	df_list.append(df)

Computing 0 out of 38, 1888 states in the Markov chain
Computing 1 out of 38, 1684 states in the Markov chain
Computing 2 out of 38, 1684 states in the Markov chain
Computing 3 out of 38, 1446 states in the Markov chain
Computing 4 out of 38, 1446 states in the Markov chain
Computing 5 out of 38, 1771 states in the Markov chain
Computing 6 out of 38, 1681 states in the Markov chain
Computing 7 out of 38, 1681 states in the Markov chain
Computing 8 out of 38, 1501 states in the Markov chain
Computing 9 out of 38, 1501 states in the Markov chain
Computing 10 out of 38, 1291 states in the Markov chain
Computing 11 out of 38, 2929 states in the Markov chain
Computing 12 out of 38, 2481 states in the Markov chain
Computing 13 out of 38, 2481 states in the Markov chain
Computing 14 out of 38, 2033 states in the Markov chain
Computing 15 out of 38, 1887 states in the Markov chain
Computing 16 out of 38, 1565 states in the Markov chain
Computing 17 out of 38, 1565 states in the Markov chain
Co

In [ ]:
df_list[1]

In [ ]:
for i in range(3):
	df = df_list[i]
	print('Table', i+1)
	print('Average absolute error for method1:', get_pct(abs(df['CR'].apply(get_num) - df['Approx CR 1'].apply(get_num)).mean()))
	print('Average absolute error for method2:', get_pct(abs(df['CR'].apply(get_num) - df['Approx CR 2'].apply(get_num)).mean()))
	print('Average error for method1:', get_pct((df['CR'].apply(get_num) - df['Approx CR 1'].apply(get_num)).mean()))
	print('Average error for method2:', get_pct((df['CR'].apply(get_num) - df['Approx CR 2'].apply(get_num)).mean()))

In [6]:
df_list[0].head()

Card note prob duration     CR Approx CR 1 Approx CR 2
0  天女绘Lv8   17   44        9  76.0%    78.6701%    75.7918%
1  天女绘Lv7   17   41        8  70.0%    69.1164%    66.8511%
2  天女绘Lv6   17   38        8  67.1%    66.2578%    64.0475%
3  天女绘Lv5   17   35        7  62.3%    64.7063%    62.4027%
4  天女绘Lv4   17   32        7  58.8%    61.1361%    58.9111%

从结果可以看出，第二种近似计算覆盖率的方法有更小的误差，本文马尔可夫链方法与《判定覆盖率和判卡培养》中动态规划的算法相比

缺点在于
* 做了过多不符合实际的预设（note到达是等间隔的，没有长note），因此不适用于冲击活动歌榜的精确计算
* 虽然可以计算每个note被覆盖的概率，但是由于前一点，note到达的时间并不是准确的谱面时间，因此没有任何意义

优势在于
* 马尔可夫链模型可以直接用于（N/P/C）分卡和奶卡的计算
* 可以考虑到不同玩家的手残程度建立不同的判定分布，
* 可以轻松扩展到C判卡（目前只有一张）以及P判卡（根本没有这种卡）的覆盖率计算，只需建立不同的马尔可夫链模型即可
* 对于T判卡马尔可夫链可以非常容易地计算覆盖时间与歌曲长度的比值，在note到达等间隔的情况下该值与覆盖率相差不大
* 与蒙特卡洛相比计算量小很多，近似算法1要求矩阵求逆复杂度为$O(n_s^3)$，近似算法2要求N次向量-矩阵乘法复杂度为$O(Nn_s^2)$，其中$n_s$为马尔可夫链中状态的个数，且$ n_s \le |\Omega| = 2K\tilde{D} $.

与动态规划算法相比，《判定覆盖率和判卡培养》4.2.1中提到分辨率为1ms，考虑到每首歌100秒左右算法需要迭代大约10万步（吐槽：不过看公式感觉每次迭代计算量很小？），而马尔可夫模型近似算法2只需迭代几百步（吐槽：别自欺欺人了，每步都要矩阵运算）。

所以最终有可能得到了一个没DP准又没DP快的算法（吐槽：我TM到底在干什么？），也许可能的用途：
* 为手残玩家计算覆盖率（因为可以考虑到Miss）
* 为穷人玩家计算覆盖率（低阶技能持续时间短，马尔可夫链状态数量少，计算会快很多）
* 等KLab出大量的C判和P判（想多了）
* 在note密度不变大的情况大歌曲长度大幅度提高，这样近似算法1准确率会提高（你够了！）

下面我们来分析C判技能，依然是马尔可夫链，与N判不同的是在判定强化的状态下，Good和Great会被强制转换成Perfect，因此连击在判定强化的状态下更有可能持续，此外每当连击断掉的时候combo次数会重新从零开始计算，设$ \alpha = p_{Perfect} + p_{Great} $, $ \beta = p_{Good} $

输入状态$(c,h,r)$, 首先计算剩余时间 $ \tilde{r} = \max(0,r-1) $

1. 无时间覆盖$\tilde{r}=0$，无累积继承$h=0$:

    * 如果未达到累积计数$c<K-1$: 
        * 如果下一个note是Good, Bad, Miss: 
        $ \Pr[(c,0,r) \to (0,0,0)] = 1-\alpha $
        * 如果下一个note不是Good, Bad, Miss: 
            $ \Pr[(c,0,r) \to (c+1,0,0)] = \alpha $
    * 如果达到累积计数$c=K-1$: 
        * 如果下一个note是Good, Bad, Miss: 
        $ \Pr[(K-1,0,r) \to (0,0,0)] = 1-\alpha $
        * 如果下一个note不是Good, Bad, Miss: 
            * 如果技能被触发: 
            $ \Pr[(K-1,0,r) \to (0,0,\tilde{D})] = p \alpha $
            * 如果技能未被触发: 
            $ \Pr[(K-1,0,r) \to (0,0,0)] = (1-p) \alpha $
        * 注意：在这个情况下断掉连击，或者维持连击并且技能触发，下一个状态都会是$(0,0,0)$，所以
        $ \Pr[(K-1,0,r) \to (0,0,0)] = 1 - p \alpha $
        
2. 无时间覆盖$\tilde{r}=0$，有累积继承$h=1$:

    * 如果下一个note是Good, Bad, Miss: 
        * 如果技能被触发:
        $ \Pr[(c,1,r) \to (0,0,\tilde{D})] = p (1-\alpha) $
        * 如果技能未被触发:
        $ \Pr[(c,1,r) \to (0,0,0)] = (1-p) (1-\alpha) $
    * 如果下一个note不是Good, Bad, Miss: 
        * 如果未达到累积计数$c<K-1$:
            * 如果技能被触发: 
            $ \Pr[(c,1,r) \to (c+1,0,\tilde{D})] = p \alpha $
            * 如果技能未被触发: 
            $ \Pr[(c,1,r) \to (c+1,0,0)] = (1-p) \alpha $
        * 如果达到累积计数$c=K-1$:
            * 如果技能被触发: 
            $ \Pr[(K-1,1,r) \to (0,1,\tilde{D})] = p \alpha $
            * 如果技能未被触发: 
            $ \Pr[(K-1,1,r) \to (0,1,0)] = (1-p) \alpha $

3. 有时间覆盖$\tilde{r}>0$，无累积继承$h=0$:
    * 如果下一个note是Bad, Miss: 
    $ \Pr[(c,0,r) \to (0,0,\tilde{r})] = 1-\alpha-\beta $
    * 如果下一个note不是Bad, Miss: 
        * 如果未达到累积计数$c<K-1$:
        $ \Pr[(c,h,r) \to (c+1,0,\tilde{r})] = \alpha+\beta $
        * 如果达到累积计数$c=K-1$:
        $ \Pr[(K-1,h,r) \to (0,1,\tilde{r})] = \alpha+\beta $
        
4. 有时间覆盖$\tilde{r}>0$，有累积继承$h=1$:
    * 如果未达到累积计数$c<K-1$:
        * 如果下一个note是Bad, Miss: 
        $ \Pr[(c,1,r) \to (0,1,\tilde{r})] = 1-\alpha-\beta $
        * 如果下一个note不是Bad, Miss: 
        $ \Pr[(c,1,r) \to (c+1,1,\tilde{r})] = \alpha+\beta $
    * 如果达到累积计数$c=K-1$:
        * 如果下一个note是Bad, Miss: 
        $ \Pr[(K-1,1,r) \to (0,1,\tilde{r})] = 1-\alpha-\beta $
        * 如果下一个note不是Bad, Miss (注:此时发生自重合损失): 
        $ \Pr[(K-1,1,r) \to (0,1,\tilde{r})] = \alpha+\beta $
        * 注意：在这个情况下不管连击是否持续，下一个状态必然是$(0,1,\tilde{r})$，所以
        $ \Pr[(K-1,1,r) \to (0,1,\tilde{r})] = 1 $
    

In [7]:
def next_state_simul(state,K,D,alpha,beta,p):
	combo, inherit, remain = state
	remain = max(remain-1,0)
	
	if remain == 0 and inherit == 0:
		if combo < K-1:
			if np.random.random() < 1-alpha:
				return (0,0,0), 1 - alpha
			else:
				return (combo+1,0,0), alpha
		elif combo == K-1:
			if np.random.random() < 1-p*alpha:
				return (0,0,0), 1 - p*alpha
			else:
				return (0,0,D), p*alpha
	elif remain == 0 and inherit == 1:
		if np.random.random() < 1-alpha:
			if np.random.random() < p:
				return (0,0,D), p*(1 - alpha)
			else:
				return (0,0,0), (1-p)*(1 - alpha)
		else:
			if combo < K-1:
				if np.random.random() < p:
					return (combo+1,0,D), p*alpha
				else:
					return (combo+1,0,0), (1-p)*alpha
			elif combo == K-1:
				if np.random.random() < p:
					return (0,1,D), p*alpha
				else:
					return (0,1,0), (1-p)*alpha
	elif remain > 0 and inherit == 0:
		if np.random.random() < 1-alpha-beta:
			return (0,0,remain), 1 - alpha - beta
		else:
			if combo < K-1:
				return (combo+1,0,remain), alpha+beta
			elif combo == K-1:
				return (0,1,remain), alpha+beta
	elif remain > 0 and inherit == 1:
		if combo < K-1:
			if np.random.random() < 1-alpha-beta:
				return (0,1,remain), 1 - alpha - beta
			else:
				return (combo+1,1,remain), alpha+beta
		else:
			return (0,1,remain), alpha+beta
			
def next_state(state,K,D,alpha,beta,p):
	combo, inherit, remain = state
	remain = max(remain-1,0)
	new_state = dict()
	if remain == 0 and inherit == 0:
		if combo < K-1:
			new_state[(0,0,0)] = 1 - alpha
			new_state[(combo+1,0,0)] = alpha
		elif combo == K-1:
			new_state[(0,0,0)] = 1 - p*alpha
			new_state[(0,0,D)] = p*alpha
	elif remain == 0 and inherit == 1:
		new_state[(0,0,D)] = p*(1 - alpha)
		new_state[(0,0,0)] = (1-p)*(1 - alpha)
		if combo < K-1:
			new_state[(combo+1,0,D)] = p*alpha
			new_state[(combo+1,0,0)] = (1-p)*alpha
		elif combo == K-1:
			new_state[(0,1,D)] = p*alpha
			new_state[(0,1,0)] = (1-p)*alpha
	elif remain > 0 and inherit == 0:
		new_state[(0,0,remain)] = 1 - alpha - beta
		if combo < K-1:
			new_state[(combo+1,0,remain)] = alpha + beta
		elif combo == K-1:
			new_state[(0,1,remain)] = alpha + beta
	elif remain > 0 and inherit == 1:
		if combo < K-1:
			new_state[(0,1,remain)] = 1 - alpha - beta
			new_state[(combo+1,1,remain)] = alpha + beta
		else:
			new_state[(0,1,remain)] = 1
	return new_state
def generate_Markov(K,D,alpha,beta,p):
	queue, trans = [(0,0,0)], dict()
	while len(queue)>0:
		state = queue.pop()
		if trans.get(state) is None:
			to_states = next_state(state,K,D,alpha,beta,p)
			trans[state] = dict()
			for st,pr in to_states.items():
				if pr >= 0:
					trans[state][st] = pr
					queue = [st]+queue
	G = nx.DiGraph()
	# Use ASCII character instead of number to make node name shorter
	for from_state, sub_dict in trans.items():
		for to_state, prob in sub_dict.items():
			G.add_edge('{0}{1}{2}'.format(*[chr(65+x) for x in from_state]), 
					   '{0}{1}{2}'.format(*[chr(65+x) for x in to_state]), prob=prob)
	M = np.array(nx.adjacency_matrix(G, weight='prob', nodelist=sorted(list(G.node))).todense(), dtype=float)
	state_list = sorted(list(G.node))
	return M, state_list

例如假设玩家打出Perfect, Great的概率之和为0.98，打出Bad的概率为0.01，并且卡牌技能简化后等价为“每3个Combo，以$40\%$的概率触发技能，每当技能被触发，那么为接下来的$4$个note加强判定”，那么对应马尔可夫链的转移矩阵将如下图所示

In [8]:
K, D_tilde, alpha, beta, p = 3, 4, 0.96, 0.02, 0.4
M, state_list = generate_Markov(K,D_tilde,alpha,beta,p)
df = pd.DataFrame(M)
df.index = [''.join([str(ord(x)-ord('A')) for x in name]) for name in state_list]
df.columns = [''.join([str(ord(x)-ord('A')) for x in name]) for name in state_list]
df.style.apply(lambda x: ["background: rgb(255,{0},128)".format(255-int(255*v)) if v > 0 else "" for v in x], axis = 1)

61号卡牌Lv8时的技能为：每17个combo，以50%的概率触发技能，加强判定6.5秒
那么在人鱼1 Master难度下：note密度6.54，722 note，这张卡的覆盖率为

In [9]:
note_density, num_note = 6.54, 722
K, D_tilde, p, alpha, beta = 17, round(note_density*6.5), 0.50, 0.98, 0.01
M, state_list = generate_Markov(K,D_tilde,alpha,beta,p)

# Compute CR by stationary distribution
ns = M.shape[0]
A = M - np.eye(ns)
A[:,-1] = 1
eq_dist = np.array([0]*(ns-1)+[1]).reshape((1,-1)) @ np.linalg.inv(A)
idx_no_cover = np.array([i for i,x in enumerate(state_list) if x[-1]=='A'])
cover_rate1 = 1 - eq_dist[0,idx_no_cover].sum()
# Compute CR by Chapman-Kolmogorov
x = np.array([1.0]+[0.0]*(ns-1)).reshape((1,-1))
cum_prob = 0
for t in range(num_note):
	cum_prob += 1-x[0,idx_no_cover].sum()
	x = x @ M
cover_rate2 = cum_prob/num_note
# Compute CR by Monte Carlo
state, total, num_note, num_trial = (0,0,0), 0, 500, 1000
for trial in range(num_trial):
	count = 0
	for i in range(num_note):
		state, prob = next_state_simul(state,K,D_tilde,alpha,beta,p)
		if state[-1] > 0:
			count +=1
	total += count/num_note
cover_rate_MC = total/num_trial

print('Approx CR1 = {0:.4f}, Approx CR2 = {1:.4f}, MC Approx CR = {2:.4f}'.format(cover_rate1, cover_rate2, cover_rate_MC))

Approx CR1 = 0.7261, Approx CR2 = 0.6994, MC Approx CR = 0.7250


可以看出，在note等间隔到达时马尔可夫链方法估计出的覆盖率与蒙特卡洛方法模拟出的结果相似，不过当前蒙特卡洛依然遵从note等间隔到达的假设，更进一步的对比需要实现一个和游戏相同的模拟器。